In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install -U sentence-transformers


from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_colwidth', None)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4e2cf6e8ae15d19e8de20567edbcce6ec365c7e936d222523ed38e635fc58f6c
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


# Datasets used
1. [Bangla Quran with tafsir](https://www.kaggle.com/datasets/mobassir/bangla-quran-with-tafsir)
2. [en_bn sahih bukhari & muslim](https://www.kaggle.com/datasets/mobassir/en-bn-sahih-bukhari-muslim) 

In [2]:
en_bn_quran_tafsir = pd.read_csv('/kaggle/input/bangla-quran-with-tafsir/en_bn_quran_tafsir.csv')

en_bn_bukhari_muslim = pd.read_csv('/kaggle/input/en-bn-sahih-bukhari-muslim/en_bn_translated_sahih_hadiths_final.csv')

In [3]:
en_bn_quran_tafsir.head(1)

,Name,Surah,Ayat,Verse,Tafseer,ar_text,আল_বায়ান,tafsir_bayan
0,The Opening,1,1,"In the name of Allah, the Beneficent, the Merciful.",In the Name of God the Compassionate the Merciful,بِسۡمِ اللّٰهِ الرَّحۡمٰنِ الرَّحِیۡمِ ﴿۱﴾\nبسم الله الرحمن الرحیم ﴿۱﴾,পরম করুণাময় অতি দয়ালু আল্লাহর নামে।,"(১) অনন্ত করুণাময় পরম দয়ালু আল্লাহর নামে (আরম্ভ করছি)।‘বিসমিল্লাহ’র পূর্বে ‘আক্বরাউ’ ‘আবদাউ’ অথবা ‘আতলু’ ফে’ল (ক্রিয়া) উহ্য আছে। অর্থাৎ, আল্লাহর নাম নিয়ে পড়ছি অথবা শুরু করছি কিংবা তেলাঅত আরম্ভ করছি। প্রত্যেক গুরুত্বপূর্ণ কাজ আরম্ভ করার পূর্বে ‘বিসমিল্লাহ’ পড়ার প্রতি তাকীদ করা হয়েছে। সুতরাং নির্দেশ করা হয়েছে যে, খাওয়া, যবেহ করা, ওযু করা এবং সহবাস করার পূর্বে ‘বিসমিল্লাহ’ পড়। অবশ্য ক্বুরআনে করীম তেলাঅত করার সময় ‘বিসমিল্লাহির রাহমানির রাহীম’ পড়ার পূর্বে ‘আউযু বিল্লাহি মিনাশ্শায়ত্বানির রাজীম’ পড়াও অত্যাবশ্যক। মহান আল্লাহ বলেছেন, ‘‘অতএব যখন তুমি ক্বুরআন পাঠ করবে, তখন বিতাড়িত শয়তান থেকে আল্লাহর আশ্রয় প্রার্থনা কর।’’ (সূরা নাহল ৯৮ আয়াত)।তাফসীরে আহসানুল বায়ান"


In [4]:
en_bn_bukhari_muslim.head(1) 

,id,hadith_id,source,chapter_no,hadith_no,chapter,chain_indx,text_ar,text_en,narrators,text_bn
0,0,1,Sahih Bukhari,1,1,Revelation - كتاب بدء الوحى,"30418, 20005, 11062, 11213, 11042, 3","حدثنا الحميدي عبد الله بن الزبير، قال حدثنا سفيان، قال حدثنا يحيى بن سعيد الأنصاري، قال أخبرني محمد بن إبراهيم التيمي، أنه سمع علقمة بن وقاص الليثي، يقول سمعت عمر بن الخطاب رضى الله عنه على المنبر قال سمعت رسول الله صلى الله عليه وسلم يقول ‏""‏ إنما الأعمال بالنيات، وإنما لكل امرئ ما نوى، فمن كانت هجرته إلى دنيا يصيبها أو إلى امرأة ينكحها فهجرته إلى ما هاجر إليه ‏""‏‏.‏","Narrated 'Umar bin Al-Khattab: I heard Allah's Apostle saying, ""The reward of deeds depends upon the intentions and every person will get the reward according to what he has intended. So whoever emigrated for worldly benefits or for a woman to marry, his emigration was for what he emigrated for.""","'Abdullah bin al-Zubair bin 'Isa عبد الله بن الزبير بن عيسى الحميدي,Sufyan bin 'Uyaynah سفيان بن عيينة,Yahya bin Sa'id al-Ansari يحيى بن سعيد الأنصاري,Muhammad bin Ibrahim bin al-Harith محمد بن إبراهيم بن الحارث,'Alqama bin Waqqas علقمة بن وقاص,'Umar ibn al-Khattab ( عمر بن الخطاب بن نفيل ( رضي الله عنه","উমর বিন আল-খাততাবের কাহিনীঃ আমি আল্লার রসূলকে বলতে শুনেছি, ""ক্রিয়া-কর্মের প্রতিদান উদ্দেশ্যের উপর নির্ভর করে এবং প্রত্যেক ব্যক্তিকে তার উদ্দেশ্য অনুযায়ী প্রতিদান দেওয়া হবে। সুতরাং যে কেউ পার্থিব জীবনোপকরণ বা বিবাহের উদ্দেশ্যে হিজরত করে, তার হিজরত তার হিজরতের জন্যই হবে।"


# part 1 - Sentence Transformers -> paraphrase-multilingual-mpnet-base-v2

In [5]:
%%time

from sentence_transformers import SentenceTransformer, util

import torch

if not torch.cuda.is_available():
    print("Warning: No GPU found. Please add GPU to your notebook")

    
#We use the Bi-Encoder to encode all passages, so that we can use it with sematic search
#paraphrase-multilingual-MiniLM-L12-v2 , paraphrase-multilingual-mpnet-base-v2
bi_encoder = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')#this one was my best model in lecr competition
bi_encoder_en = SentenceTransformer('all-mpnet-base-v2')#english only

print(bi_encoder.max_seq_length)
bi_encoder.max_seq_length = 512     #Truncate long passages to 256 tokens
top_k = 32                          #Number of passages we want to retrieve with the bi-encoder

passages_bn = en_bn_quran_tafsir['tafsir_bayan']  #আল_বায়ান ,tafsir_bayan 
print("Passages:", len(passages_bn))

corpus_embeddings = bi_encoder.encode(passages_bn, convert_to_tensor=True, show_progress_bar=True)
np.save("holy_quran_bn_emb_paraphrase-multilingual-mpnet-base-v2.npy",corpus_embeddings.cpu().numpy())
corpus_embeddings.shape

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/723 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

128
Passages: 6236


Batches:   0%|          | 0/195 [00:00<?, ?it/s]

CPU times: user 1min 20s, sys: 6.86 s, total: 1min 27s
Wall time: 1min 35s


torch.Size([6236, 768])

In [6]:
%%time

passages = en_bn_quran_tafsir['Verse'] + " "+en_bn_quran_tafsir['Tafseer']
print("Passages:", len(passages))

corpus_embeddings_en = bi_encoder_en.encode(passages, convert_to_tensor=True, show_progress_bar=True)
np.save("holy_quran_emb_all-mpnet-base-v2.npy",corpus_embeddings_en.cpu().numpy())
corpus_embeddings.shape

Passages: 6236


Batches:   0%|          | 0/195 [00:00<?, ?it/s]

CPU times: user 25.2 s, sys: 158 ms, total: 25.3 s
Wall time: 24.2 s


torch.Size([6236, 768])

In [7]:
#using mlt embed

query = "is god one and only?"
top_k = 5
# Encode the query using the bi-encoder and find potentially relevant wikitexts

question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
question_embedding = question_embedding.cuda()
hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=top_k)
print(hits)
hits = hits[0]  # Get the hits for the first query

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'corpus_id': 113, 'score': 0.7093433737754822}, {'corpus_id': 3494, 'score': 0.7036828994750977}, {'corpus_id': 2658, 'score': 0.6971420645713806}, {'corpus_id': 481, 'score': 0.6940592527389526}, {'corpus_id': 3496, 'score': 0.6632385849952698}]]


In [8]:
# Output of top-3 hits from bi-encoder
print("\n-------------------------\n")
print("Top-3 Bi-Encoder Retrieval hits\n\n\n")
hits = sorted(hits, key=lambda x: x['score'], reverse=True)
for hit in hits[0:3]:
    print("\t{:.3f}\t{}".format(hit['score'], passages_bn[hit['corpus_id']].replace("\n", " ")))
    print("\n\n\n")


-------------------------

Top-3 Bi-Encoder Retrieval hits



	0.709	১০৭। তুমি কি জান না যে, আকাশমন্ডলী ও পৃথিবীর সার্বভৌমত্ব একমাত্র আল্লাহরই? আর আল্লাহ ছাড়া তোমাদের কোন অভিভাবক নেই এবং সাহায্যকারীও নেই।-তাফসীরে আহসানুল বায়ান




	0.704	(২৬) আকাশমন্ডলী ও পৃথিবীতে যা কিছু আছে, তা আল্লাহরই।[1] নিশ্চয়ই আল্লাহ অভাবমুক্ত,[2] প্রশংসার্হ। [3][1] অর্থাৎ, সে সবের সৃষ্টিকর্তাও তিনি, মালিকও তিনি এবং বিশ্ব-জগতের পরিচালকও তিনি।[2] সকল কিছু হতে অমুখাপেক্ষী। অর্থাৎ, সকল সৃষ্টি তাঁর মুখাপেক্ষী এবং তিনি কারো মুখাপেক্ষী নন।[3] তাঁর সকল প্রকার সৃষ্ট বস্তুতে। সুতরাং তিনি যা কিছু সৃষ্টি করেছেন এবং যে আহ্কাম অবতীর্ণ করেছেন, তার উপর আকাশ ও পৃথিবীর সকল প্রশংসার অধিকারী একমাত্র তিনিই।তাফসীরে আহসানুল বায়ান




	0.697	(৬৪) আকাশমন্ডলী ও পৃথিবীতে যা কিছু আছে তা তাঁরই।[1] আর নিশ্চয় আল্লাহ; তিনিই অভাবমুক্ত, সর্বপ্রশংসনীয়।[1] সৃষ্টির দিক দিয়ে, মালিকানার দিক দিয়ে এবং নিয়ন্ত্রণ ও পরিচালনার দিক দিয়ে। কারণ সকল সৃষ্টি তাঁর মুখাপেক্ষী এবং তিনি কারো মুখাপেক্ষী নন। যেহেতু তিনি অভাবশূন্য অমুখাপেক্ষী। সেই সত্তা সকল পরিপূর্

In [9]:
#using mlt embed

query = "who is a good muslim?"
top_k = 5
# Encode the query using the bi-encoder and find potentially relevant wikitexts

question_embedding = bi_encoder_en.encode(query, convert_to_tensor=True)
question_embedding = question_embedding.cuda()
hits = util.semantic_search(question_embedding, corpus_embeddings_en, top_k=top_k)
print(hits)
hits = hits[0]  # Get the hits for the first query

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'corpus_id': 3398, 'score': 0.6377196311950684}, {'corpus_id': 1942, 'score': 0.5826699137687683}, {'corpus_id': 1426, 'score': 0.5812615156173706}, {'corpus_id': 561, 'score': 0.5262311697006226}, {'corpus_id': 1983, 'score': 0.5088825821876526}]]


In [10]:
# Output of top-3 hits from bi-encoder
print("\n-------------------------\n")
print("Top-3 Bi-Encoder Retrieval hits\n\n\n")
hits = sorted(hits, key=lambda x: x['score'], reverse=True)
for hit in hits[0:3]:
    print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))
    print("\n\n\n")
    print("\t{:.3f}\t{}".format(hit['score'], passages_bn[hit['corpus_id']].replace("\n", " ")))
    print("\n\n\n")


-------------------------

Top-3 Bi-Encoder Retrieval hits



	0.638	Who persevere, and put their trust in their Lord! they are those who are steadfast in enduring the harm inflicted upon them by the idolaters and in enduring the Emigration in order to make manifest the religion of Islam and who put their trust in their Lord so that He provides for them whence they do not expect.




	0.638	(৫৯) যারা ধৈর্য অবলম্বন করে[1] ও তাদের প্রতিপালকের ওপর নির্ভর করে। [2][1] অর্থাৎ, দ্বীনের উপর সুদৃঢ়ভাবে প্রতিষ্ঠিত থাকে, হিজরতের কষ্ট বরণ করে, পরিবার-পরিজন, আত্মীয়-স্বজন ও বন্ধু-বান্ধব থেকে দূরে থাকার কষ্টকে একমাত্র আল্লাহর সন্তষ্টি লাভের জন্য মেনে নেয়।[2] দ্বীন ও দুনিয়ার সকল বিষয়ে এবং সর্বাবস্থায়।তাফসীরে আহসানুল বায়ান




	0.583	Such as are steadfast and put their trust in Allah. They are those who endure the harm inflicted by the idolaters and endure the emigration in order to make manifest the religion and put their trust in their Lord so that He provides for them whence they do not reckon.





# part 2 - faiss indexing with paraphrase-xlm-r-multilingual-v1

ref -> https://towardsdatascience.com/how-to-build-a-semantic-search-engine-with-transformers-and-faiss-dcbea307a0e8

In [11]:
!pip install faiss-cpu

# Used to import data from S3.
import pandas as pd


# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer

# Used to create and store the Faiss index.
import faiss
import numpy as np
import pickle

# Used to do vector searches and display the results.


def vector_search(query, model, index, num_results=10):
    """Tranforms query to vector using a pretrained, sentence-level 
    DistilBERT model and finds similar vectors using FAISS.
    Args:
        query (str): User query that should be more than a sentence long.
        model (sentence_transformers.SentenceTransformer.SentenceTransformer)
        index (`numpy.ndarray`): FAISS index that needs to be deserialized.
        num_results (int): Number of results to return.
    Returns:
        D (:obj:`numpy.array` of `float`): Distance between results and query.
        I (:obj:`numpy.array` of `int`): Paper ID of the results.
    
    """
    vector = model.encode(list(query))
    D, I = index.search(np.array(vector).astype("float32"), k=num_results)
    return D, I


def id2details(df, I, column):
    """Returns the paper titles based on the paper index."""
    return [list(df[df.id == idx][column]) for idx in I[0]]



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 59.4 MB/s eta 0:00:00


# quran

In [12]:
%%time
df = en_bn_quran_tafsir
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

# Check if CUDA is available ans switch to GPU
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)
print(model.max_seq_length)
# Convert text to vectors
embeddings = model.encode(df.tafsir_bayan.to_list(), show_progress_bar=True)
np.save("bn_quran_paraphrase-xlm-r-multilingual-v1.npy",embeddings)
embeddings.shape

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

cuda:0
128


Batches:   0%|          | 0/195 [00:00<?, ?it/s]

CPU times: user 49.7 s, sys: 4.08 s, total: 53.8 s
Wall time: 55 s


(6236, 768)

In [13]:

# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
df.id = 0
ids = np.arange(embeddings.shape[0]) 
df.id = ids 
# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.id)

# Retrieve the 2 nearest neighbours
D, I = index.search(np.array([embeddings[77]]), k=2)
print(f"L2 dist {D.flatten().tolist()} \n\nid {I.flatten().tolist()}")

L2 dist [0.0, 5.648537635803223] 

id [77, 1516]


In [14]:
id2details(df, I, "tafsir_bayan")

[["৭১। মূসা বলল, তিনি বলছেন, ‘এ এমন একটি গাভী যা জমির চাষে ও ক্ষেতে পানি সেচের জন্য ব্যবহূত হয়নি -- সুস্থ নিখুঁত।’ তারা বলল, ‘এখন তুমি সঠিক বর্ণনা এনেছ।’ অতঃপর তারা তা যবেহ করল, অথচ যবেহ করতে পারবে বলে মনে হচ্ছিল না। (1)(1) তাদেরকে এই নির্দেশই দেওয়া হয়েছিল যে, তারা একটি গাভী যবেহ করবে। তারা যে কোন একটি গাভী যবেহ করলেই আল্লাহর আদেশ পালন হয়ে যেত। কিন্তু তারা আল্লাহর নির্দেশের উপর সোজাসুজি আমল করার পরিবর্তে খুঁটিনাটির পিছনে পড়ে বিভিন্ন রকমের প্রশ্ন করতে শুরু করে দিল। ফলে আল্লাহ তাআলাও তাদের সে কাজকে পর্যায়ক্রমে তাদের জন্য কঠিন করে দিলেন। আর এই জন্যই দ্বীনের (খুঁটিনাটির) ব্যাপারে গভীরভাবে অনুসন্ধান চালাতে ও কঠিনতা অবলম্বন করতে নিষেধ করা হয়েছে। (প্রকাশ যে, এই গাভী যবেহর ঘটনার উল্লেখের ফলেই এই সূরার নাম 'বাক্বারাহ' রাখা হয়েছে।)তাফসীরে আহসানুল বায়ান"],
 ['(৪৪) আর বলা হল, ‘হে ভূমি! তুমি নিজ পানি শুষে নাও[1] এবং হে আকাশ! তুমি ক্ষান্ত হও।’ তখন পানি কমে গেল ও নির্ধারিত কার্য সম্পন্ন হল।[2] নৌকা জুদী (পাহাড়) [3]-এর উপর এসে থামল। আর বলা হল, ‘অন্যায়কারীরা আল্লাহর করুণা হতে দূর হোক।’ [4][1] ابلعي ‘গ

In [15]:
user_query = "i am frustrated"
# Querying the index
D, I = vector_search([user_query], model, index, num_results=5)

id2details(df, I, "tafsir_bayan")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[['(৩৬) আমি যখন মানুষকে অনুগ্রহ আস্বাদ করাই, তখন ওরা তাতে উৎফুল্ল হয় এবং নিজেদের কৃতকর্মের ফলে ওরা দুর্দশাগ্রস্ত হলেই ওরা হতাশ হয়ে পড়ে। [1][1] এটাও ঐ একই বিষয়, যা সূরা হূদের ৯-১০নং আয়াতে আলোচিত হয়েছে। অধিকাংশ মানুষের অভ্যাস এই যে, সুখের সময় তারা গর্বিত হয় এবং দুর্দশার সময় হতাশ হয়ে পড়ে। তবে মু’মিনগণ এ ব্যাপারে পৃথক। তারা দুঃখ-কষ্টে ধৈর্যধারণ করে এবং সুখের সময় আল্লাহর কৃতজ্ঞতা প্রকাশ করে। অর্থাৎ নেক আমল করে। অবশ্য তাদের জন্য উভয় অবস্থাই মঙ্গল ও নেকী উপার্জনের কারণ হয়ে যায়।তাফসীরে আহসানুল বায়ান'],
 ['১৯। পক্ষান্তরে যারা আমার নিদর্শন প্রত্যাখ্যান করেছে, তারাই হল হতভাগ্য।-তাফসীরে আহসানুল বায়ান'],
 ['(৯৪) আমি কোন জনপদে নবী পাঠালে ওর অধিবাসীবৃন্দকে দুঃখ ও ক্লেশ দ্বারা পীড়িত করি, যাতে তারা কাকুতি-মিনতি করে।[1][1] بأسَاء শারীরিক কষ্টকে বুঝায়; যেমন অসুখ ও অসুস্থতা। আর ضَرَّاء বলা হয় অভাব ও দারিদ্র্যকে। উদ্দেশ্য এই যে, যে জনপদেই আমি রসূল প্রেরণ করি এবং সেখানকার মানুষ তাকে মিথ্যাজ্ঞান করে, যার ফলে আমি তাদেরকে অসুখ ও দারিদ্র্য দিয়ে পরীক্ষা করে থাকি, যার উদ্দেশ্য হয় যে, তারা যেন আল্লাহর পথে ফিরে আ

# hadith

In [16]:
%%time
embeddings = model.encode(en_bn_bukhari_muslim.text_en.to_list(), show_progress_bar=True)
np.save("bn_hadith_paraphrase-xlm-r-multilingual-v1.npy",embeddings)

Batches:   0%|          | 0/468 [00:00<?, ?it/s]

CPU times: user 57.7 s, sys: 312 ms, total: 58 s
Wall time: 52.9 s


In [17]:
df = en_bn_bukhari_muslim
# Step 1: Change data type
embeddings = np.array([embedding for embedding in embeddings]).astype("float32")
df.id = 0
ids = np.arange(embeddings.shape[0]) 
df.id = ids 
# Step 2: Instantiate the index
index = faiss.IndexFlatL2(embeddings.shape[1])

# Step 3: Pass the index to IndexIDMap
index = faiss.IndexIDMap(index)

# Step 4: Add vectors and their IDs
index.add_with_ids(embeddings, df.id)

# Retrieve the 3 nearest neighbours
D, I = index.search(np.array([embeddings[77]]), k=3)
print(f"L2 dist {D.flatten().tolist()} \n\nid {I.flatten().tolist()}")

L2 dist [0.0, 2.806145191192627, 3.3565173149108887] 

id [77, 73, 3262]


In [18]:
user_query = "who is a better muslim?"
# Querying the index
D, I = vector_search([user_query], model, index, num_results=5)

print(f"L2 dist {D.flatten().tolist()} \n\nid {I.flatten().tolist()}")
id2details(df, I, "text_bn")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

L2 dist [26.856534957885742, 27.35055160522461, 27.460647583007812, 27.94554901123047, 28.042133331298828] 

id [11810, 10, 5726, 3957, 14915]


[['আর রওয়া লিথও আবদুর রহমান বিন খালেদ বিন মোজাহার, ইবন শাহাব, এই ভিত্তিতে।'],
 ['আবু মূসা (আঃ) এর কাহিনীঃ কিছু লোক রাসূলুল্লাহ (সাঃ) কে জিজ্ঞেস করেছিল, "কাদের ইসলাম সবচেয়ে ভালো? তিনি বললেন, "যে ব্যক্তি জিহ্বা ও হাত দিয়ে মুসলমানদের ক্ষতি করতে এড়ায়। "'],
 ['অথবা জাবর বিন আব্দুল্লাহ ছিলেন মহানবী (সাঃ) এর বিশাল হাত ও পা, আমি তার পরে তার মতো আর কাউকে দেখিনি।'],
 ['আর ইব্রাহীম ইবনে তাহমান শিবানীর উপর, ইদীর উপর, বীরের উপর, ইবন আজাবের উপর।'],
 ['আর আমরু নাকতিকে, হাশম বিন কাসিমকে, আবু বকর বিন আবী শিবাকে, যুবতী বিন সুয়ারকে, দুজনেই শিবিরের, এই ভিত্তিতে।']]

In [19]:
# Serialise index and store it as a pickle
with open("./faiss_index.pickle", "wb") as h:
    pickle.dump(faiss.serialize_index(index), h)

In [20]:

df.iloc[10]

id                                                                                                                                                                                                                                                                                                                    10
hadith_id                                                                                                                                                                                                                                                                                                             11
source                                                                                                                                                                                                                                                                                                    Sahih Bukhari 
chapter_no                                                   